# Command-Line Interface

> Command-line tools for barcode extraction and processing

## Introduction
This notebook implements the command-line interface (CLI) for BarcodeSeqKit, allowing users to easily extract barcodes from BAM and FASTQ files without writing Python code.

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import re
import sys
import logging
import argparse
from typing import List, Dict, Tuple, Optional, Union
import yaml

from BarcodeSeqKit.core import (
    BarcodeConfig, 
    BarcodeLocationType,
    BarcodeExtractorConfig,
    FileFormat
)
from BarcodeSeqKit.bam_processing import process_bam_file
from BarcodeSeqKit.fastq_processing import process_fastq_files

## Command-Line Argument Parser
Let's define the argument parser for the command-line interface.

In [ ]:
#| export
def add_extract_arguments(parser: argparse.ArgumentParser):
    """Add arguments for the extract command.
    
    Args:
        parser: ArgumentParser to add arguments to
    """
    # Input files (required)
    input_group = parser.add_argument_group('Input options')
    
    # Input can be specified in different ways
    input_mutex = input_group.add_mutually_exclusive_group(required=True)
    
    input_mutex.add_argument(
        '--bam',
        help='Input BAM file'
    )
    
    input_mutex.add_argument(
        '--fastq1',
        help='Input FASTQ file (read 1 for paired-end data)'
    )
    
    input_mutex.add_argument(
        '--fastq-dir',
        help='Directory containing paired FASTQ files'
    )
    
    # When specifying fastq1, fastq2 is optional
    input_group.add_argument(
        '--fastq2',
        help='Input FASTQ file (read 2 for paired-end data, required if --fastq1 is used)'
    )
    
    # Barcode configuration (required)
    barcode_group = parser.add_argument_group('Barcode options')
    
    barcode_mutex = barcode_group.add_mutually_exclusive_group(required=True)
    
    barcode_mutex.add_argument(
        '--barcode-config',
        help='YAML file containing barcode configurations'
    )
    
    barcode_mutex.add_argument(
        '--barcode',
        help='Single barcode sequence (no specific location, will generate barcode_orientFR and barcode_orientRC outputs)'
    )
    
    barcode_mutex.add_argument(
        '--barcode5',
        help="5' barcode sequence (will generate barcode5_orientFR and barcode5_orientRC outputs)"
    )
    
    barcode_group.add_argument(
        '--barcode3',
        help="3' barcode sequence (will generate barcode3_orientFR and barcode3_orientRC outputs, can be used with --barcode5)"
    )
    
    barcode_group.add_argument(
        '--max-mismatches',
        type=int,
        default=0,
        help='Maximum number of mismatches allowed in barcode detection (default: 0)'
    )
    
    # Output options
    output_group = parser.add_argument_group('Output options')
    
    output_group.add_argument(
        '--output-prefix',
        required=True,
        help='Prefix for output files'
    )
    
    output_group.add_argument(
        '--output-dir',
        default='.',
        help='Directory for output files (default: current directory)'
    )
    
    # Add the new option below the other output options
    output_group.add_argument(
        '--only-stats',
        action='store_true',
        help='Generate only statistics without writing output sequence files'
    )   
    
    
    # Processing options for BAM files
    bam_group = parser.add_argument_group('BAM-specific options')
    
    bam_group.add_argument(
        '--search-softclipped',
        action='store_true',
        default=False,
        help='Search for barcodes in softclipped read region (BAM only)'
    )
    
    # Processing options for FASTQ files
    fastq_group = parser.add_argument_group('FASTQ-specific options')
    
    fastq_group.add_argument(
        '--search-both-reads',
        action='store_true',
        default=True,
        help='Search for barcodes in both reads of paired-end data (FASTQ only)'
    )
    
    fastq_group.add_argument(
        '--no-compress',
        action='store_true',
        help='Do not compress output FASTQ files (FASTQ only)'
    )
    
    # Logging options
    logging_group = parser.add_argument_group('Logging options')
    
    logging_group.add_argument(
        '--verbose',
        action='store_true',
        help='Enable verbose logging'
    )
    
    logging_group.add_argument(
        '--log-file',
        help='Log file path'
    )
    
    return parser

## Main Entry Points

In [ ]:
#| export
def run_cli(args: Optional[List[str]] = None) -> int:
    """Handle the extract command.
    
    Args:
        args: Command-line arguments
        
    Returns:
        Exit code (0 for success, non-zero for error)
    """
    parser = argparse.ArgumentParser(
        description="BarcodeSeqKit: A toolkit for barcode extraction from sequencing data",
        formatter_class=argparse.RawDescriptionHelpFormatter
    )   
    
    parser = add_extract_arguments(parser)
    
    try:
        parsed_args = parser.parse_args(args)
        
        # Create output directory if it doesn't exist
        os.makedirs(parsed_args.output_dir, exist_ok=True)
        
        # Determine the input file format and paths
        input_files = []
        
        if parsed_args.bam:
            input_files = [parsed_args.bam]
            file_format = FileFormat.BAM
            print(f"Input BAM file: {parsed_args.bam}")
        elif parsed_args.fastq1:
            if parsed_args.fastq2:
                input_files = [parsed_args.fastq1, parsed_args.fastq2]
                print(f"Input FASTQ files: {parsed_args.fastq1}, {parsed_args.fastq2}")
            else:
                input_files = [parsed_args.fastq1]
                print(f"Input FASTQ file: {parsed_args.fastq1}")
            file_format = FileFormat.FASTQ
        elif parsed_args.fastq_dir:
            input_files = [parsed_args.fastq_dir]
            file_format = FileFormat.FASTQ
            print(f"Input FASTQ directory: {parsed_args.fastq_dir}")
        
        # Load or create barcode configurations
        barcodes = []
        
        if parsed_args.barcode_config:
            # Load from YAML file
            with open(parsed_args.barcode_config, 'r') as f:
                config_data = yaml.safe_load(f)
            
            if 'barcodes' in config_data:
                for barcode_data in config_data['barcodes']:
                    location = BarcodeLocationType(barcode_data.get('location', 'UNK'))
                    barcodes.append(BarcodeConfig(
                        sequence=barcode_data['sequence'],
                        location=location,
                        name=barcode_data.get('name'),
                        description=barcode_data.get('description')
                    ))
            else:
                print("Error: Invalid barcode configuration file")
                return 1
        else:
            # Create from command-line arguments
            if parsed_args.barcode:
                # Single barcode mode (no specific location)
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode,
                    location=BarcodeLocationType.UNKNOWN,
                    name="generic",
                    description="Generic barcode from command line"
                ))
                print(f"Using single barcode mode with sequence: {parsed_args.barcode}")
            elif parsed_args.barcode5:
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode5,
                    location=BarcodeLocationType.FIVE_PRIME,
                    name="5prime",
                    description="5' barcode from command line"
                ))
                print(f"Using 5' barcode with sequence: {parsed_args.barcode5}")
                
                if parsed_args.barcode3:
                    barcodes.append(BarcodeConfig(
                        sequence=parsed_args.barcode3,
                        location=BarcodeLocationType.THREE_PRIME,
                        name="3prime",
                        description="3' barcode from command line"
                    ))
                    print(f"Using 3' barcode with sequence: {parsed_args.barcode3}")
            elif parsed_args.barcode3:
                # If only barcode3 is provided without barcode5, treat as single barcode
                barcodes.append(BarcodeConfig(
                    sequence=parsed_args.barcode3,
                    location=BarcodeLocationType.UNKNOWN,
                    name="generic",
                    description="Generic barcode from command line (provided as barcode3)"
                ))
                print(f"Using single barcode mode with sequence: {parsed_args.barcode3} (from barcode3)")
        
        # Validate that we have at least one barcode
        if not barcodes:
            print("Error: No barcode configurations provided")
            return 1
        
        # Create extractor configuration
        config = BarcodeExtractorConfig(
            barcodes=barcodes,
            output_prefix=parsed_args.output_prefix,
            output_dir=parsed_args.output_dir,
            max_mismatches=parsed_args.max_mismatches,
            search_softclipped=parsed_args.search_softclipped if file_format == FileFormat.BAM else False,
            verbose=parsed_args.verbose,
            log_file=parsed_args.log_file,
            write_output_files=not parsed_args.only_stats  # Set to False if only_stats flag is present
        )
        
        # Save the configuration for reproducibility
        config_path = os.path.join(parsed_args.output_dir, f"{parsed_args.output_prefix}_config.yaml")
        config.save_yaml(config_path)
        print(f"Saved configuration to {config_path}")
        
        # Process files based on format
        if file_format == FileFormat.BAM:
            process_bam_file(config, input_files[0])
        elif file_format == FileFormat.FASTQ:
            process_fastq_files(
                config=config,
                fastq_files=input_files,
                compress_output=not parsed_args.no_compress,
                search_both_reads=parsed_args.search_both_reads
            )
        
        print("Extraction complete")
        return 0
        
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return 1

In [ ]:
#| export
def main() -> int:
    """Main entry point for command-line execution."""
    # Only actually call sys.exit() when running as a script, not in tests or notebooks
    is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
    # Use test arguments when in test/notebook environment
    if is_notebook_or_test:
        # For testing, you can add test code here
        return 0
    else:
        return run_cli()

In [ ]:
#| eval: false
#| export
if __name__ == "__main__":
    # For safety, wrap in try/except to catch any SystemExit
    try:
        exit_code = main()
        is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
        if is_notebook_or_test:
            print(f"\nCLI completed with exit code: {exit_code}")
    except SystemExit as e:
        # Just in case, catch any SystemExit and print instead
        is_notebook_or_test = 'ipykernel' in sys.modules or 'pytest' in sys.modules or 'NBDEV_TEST' in os.environ
        if is_notebook_or_test:
            print(f"\nSystemExit caught with code: {e.code}")
        else:
            raise            


CLI completed with exit code: 0


In [ ]:
# Testing the CLI with example arguments
# Note: In a real environment, these would be passed via the command line
# Define test arguments 
test_args = [
    "--bam", "../tests/test.bam",
    "--barcode5", "CTGACTCCTTAAGGGCC",
    "--barcode3", "TAACTGAGGCCGGC",
    "--output-prefix", "test_cli_out",
    "--output-dir", "../tests/cli_output",
    "--search-softclipped",
    "--verbose"
]

# Uncomment to run the test
return_code = run_cli(test_args)
print(f"CLI test returned: {return_code}")

2025-03-24 13:59:31,992 - BarcodeSeqKit - INFO - BAM file: ../tests/test.bam (498 reads)
2025-03-24 13:59:31,993 - BarcodeSeqKit - INFO - Output categories: ['barcode5_orientFR', 'barcode5_orientRC', 'barcode3_orientFR', 'barcode3_orientRC', 'noBarcode']


Input BAM file: ../tests/test.bam
Using 5' barcode with sequence: CTGACTCCTTAAGGGCC
Using 3' barcode with sequence: TAACTGAGGCCGGC
Saved configuration to ../tests/cli_output/test_cli_out_config.yaml


Classifying reads:   0%|          | 0/498 [00:00<?, ?it/s]

2025-03-24 13:59:32,029 - BarcodeSeqKit - INFO - First pass complete: classified 18 reads


Writing reads:   0%|          | 0/498 [00:00<?, ?it/s]

2025-03-24 13:59:32,056 - BarcodeSeqKit - INFO - Sorting and indexing ../tests/cli_output/test_cli_out_barcode5_orientFR.bam
2025-03-24 13:59:32,068 - BarcodeSeqKit - INFO - Sorting and indexing ../tests/cli_output/test_cli_out_barcode5_orientRC.bam
2025-03-24 13:59:32,077 - BarcodeSeqKit - INFO - Sorting and indexing ../tests/cli_output/test_cli_out_barcode3_orientFR.bam
2025-03-24 13:59:32,085 - BarcodeSeqKit - INFO - Sorting and indexing ../tests/cli_output/test_cli_out_barcode3_orientRC.bam
2025-03-24 13:59:32,093 - BarcodeSeqKit - INFO - Sorting and indexing ../tests/cli_output/test_cli_out_noBarcode.bam


Extraction complete
CLI test returned: 0


In [ ]:
# Testing the CLI with only-stats
# Note: In a real environment, these would be passed via the command line
# Define test arguments 
test_args = [
    "--bam", "../tests/test.bam",
    "--barcode5", "CTGACTCCTTAAGGGCC",
    "--barcode3", "TAACTGAGGCCGGC",
    "--output-prefix", "test_cli_out",
    "--output-dir", "../tests/cli_output_only_stats",
    "--search-softclipped",
    "--only-stats",
    "--verbose"
]

# Uncomment to run the test
return_code = run_cli(test_args)
print(f"CLI test returned: {return_code}")

2025-03-24 14:02:05,410 - BarcodeSeqKit - INFO - BAM file: ../tests/test.bam (498 reads)
2025-03-24 14:02:05,412 - BarcodeSeqKit - INFO - Output categories: ['barcode5_orientFR', 'barcode5_orientRC', 'barcode3_orientFR', 'barcode3_orientRC', 'noBarcode']


Input BAM file: ../tests/test.bam
Using 5' barcode with sequence: CTGACTCCTTAAGGGCC
Using 3' barcode with sequence: TAACTGAGGCCGGC
Saved configuration to ../tests/cli_output_only_stats/test_cli_out_config.yaml


Classifying reads:   0%|          | 0/498 [00:00<?, ?it/s]

2025-03-24 14:02:05,456 - BarcodeSeqKit - INFO - First pass complete: classified 18 reads


Extraction complete
CLI test returned: 0


## Command-Line Examples

Here are some example command-line invocations for reference:


### Example 1: Extract barcodes from a BAM file

```bash
barcodeseqkit --bam tests/test.bam --barcode5 CTGACTCCTTAAGGGCC --output-prefix barcode_extraction --output-dir results
```

### Example 2: Extract barcodes from paired FASTQ files

```bash
barcodeseqkit --fastq1 tests/test.1.fastq.gz --fastq2 tests/test.2.fastq.gz --barcode GCCTCGCGA --output-prefix fastq_results --output-dir results
```

### Extract 5' and 3' barcodes from a directory containing FASTQ files

```bash
barcodeseqkit --fastq-dir ./fastq_dir --barcode5 ACTGACTG --barcode3 GTCAGTCA --output-prefix sample_barcoded --output-dir ./output --search-both-reads
```

### Use a barcode configuration file

```bash
barcodeseqkit --bam tests/test.bam --barcode-config barcode_config.yaml --output-prefix config_extraction --output-dir results
```

## Barcode Configuration YAML Example

Here's an example of a barcode configuration YAML file:

```yaml
barcodes:
  - sequence: CTGACTCCTTAAGGGCC
    location: 5
    name: 5prime
    description: 5' barcode for my experiment
  - sequence: TAACTGAGGCCGGC
    location: 3
    name: 3prime
    description: 3' barcode for my experiment
```

## Conclusion
This notebook implements a command-line interface for BarcodeSeqKit, making it easy to use the library's functionality without writing Python code. The CLI provides access to all the major features of the library, including barcode extraction from BAM and FASTQ files, customization options for searching and output, and comprehensive logging.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()